## Tensorflow2实现推荐系统双塔DNN排序

演示步骤：
1. 读取电影数据集，包括（用户信息、电影信息、评分行为信息）
2. 搭建双塔模型并训练，用到了keras函数Embedding、点积等技术
3. 模型应用1：保存模型用于在线predict预估
4. 模型应用2：导出embedding用于召回

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [2]:
tf.__version__

'2.2.0'

### 1. 读取和处理数据

In [3]:
df_user = pd.read_csv("./datas/ml-1m/users.dat",
                     sep="::", header=None, engine="python",
                     names = "UserID::Gender::Age::Occupation::Zip-code".split("::"))

df_movie = pd.read_csv("./datas/ml-1m/movies.dat",
                     sep="::", header=None, engine="python",
                     names = "MovieID::Title::Genres".split("::"))

df_rating = pd.read_csv("./datas/ml-1m/ratings.dat",
                     sep="::", header=None, engine="python",
                     names = "UserID::MovieID::Rating::Timestamp".split("::"))

In [4]:
df_rating.to_csv("./datas/ml-latest-small/ratings_1m.csv", index=False)

#### 只取频率最高的电影分类

In [4]:
import collections

In [5]:
# 计算电影中每个题材的次数
genre_count = collections.defaultdict(int)
for genres in df_movie["Genres"].str.split("|"):
    for genre in genres:
        genre_count[genre] += 1
genre_count

defaultdict(int,
            {'Animation': 105,
             "Children's": 251,
             'Comedy': 1200,
             'Adventure': 283,
             'Fantasy': 68,
             'Romance': 471,
             'Drama': 1603,
             'Action': 503,
             'Crime': 211,
             'Thriller': 492,
             'Horror': 343,
             'Sci-Fi': 276,
             'Documentary': 127,
             'War': 143,
             'Musical': 114,
             'Mystery': 106,
             'Film-Noir': 44,
             'Western': 68})

In [6]:
# 只保留最有代表性的题材
def get_highrate_genre(x):
    sub_values = {}
    for genre in x.split("|"):
        sub_values[genre] = genre_count[genre]
    return sorted(sub_values.items(), key=lambda x:x[1], reverse=True)[0][0]
df_movie["Genres"] = df_movie["Genres"].map(get_highrate_genre)

In [7]:
df_movie.sample(frac=1).head(3)

,MovieID,Title,Genres
1023,1036,Die Hard (1988),Action
350,354,Cobb (1994),Drama
376,380,True Lies (1994),Comedy


#### 给列新增数字索引列

目的是：防止embedding过大

In [8]:
def add_index_column(param_df, column_name):
    values = list(param_df[column_name].unique())
    value_index_dict = {value:idx for idx,value in enumerate(values)}
    param_df[f"{column_name}_idx"] = param_df[column_name].map(value_index_dict)

In [9]:
add_index_column(df_user, "UserID")
add_index_column(df_user, "Gender")
add_index_column(df_user, "Age")
add_index_column(df_user, "Occupation")
add_index_column(df_movie, "MovieID")
add_index_column(df_movie, "Genres")

In [37]:
df_user.to_csv("./datas/ml-latest-small/tensorflow_user_datawithindex.csv", index=False)
df_user.head()

,UserID,Gender,Age,Occupation,Zip-code,UserID_idx,Gender_idx,Age_idx,Occupation_idx
0,1,F,1,10,48067,0,0,0,0
1,2,M,56,16,70072,1,1,1,1
2,3,M,25,15,55117,2,1,2,2
3,4,M,45,7,02460,3,1,3,3
4,5,M,25,20,55455,4,1,2,4


In [38]:
df_movie.to_csv("./datas/ml-latest-small/tensorflow_movie_datawithindex.csv", index=False)
df_movie.head()

,MovieID,Title,Genres,MovieID_idx,Genres_idx
0,1,Toy Story (1995),Comedy,0,0
1,2,Jumanji (1995),Adventure,1,1
2,3,Grumpier Old Men (1995),Comedy,2,0
3,4,Waiting to Exhale (1995),Drama,3,2
4,5,Father of the Bride Part II (1995),Comedy,4,0


#### 合并成一个DF

In [12]:
# 合并成一个df
df = pd.merge(pd.merge(df_rating, df_user), df_movie)
df.drop(columns=["Timestamp", "Zip-code", "Title"], inplace=True)

In [13]:
df.sample(frac=1).head(3)

,UserID,MovieID,Rating,Gender,Age,Occupation,UserID_idx,Gender_idx,Age_idx,Occupation_idx,Genres,MovieID_idx,Genres_idx
901251,6040,1211,5,M,25,6,6039,1,2,20,Drama,1193,2
841741,267,3831,5,M,45,12,266,1,3,7,Comedy,3761,0
22734,5459,2321,5,F,1,10,5458,0,0,0,Comedy,2252,0


In [14]:
num_users = df["UserID_idx"].max() + 1
num_movies = df["MovieID_idx"].max() + 1
num_genders = df["Gender_idx"].max() + 1
num_ages = df["Age_idx"].max() + 1
num_occupations = df["Occupation_idx"].max() + 1
num_genres = df["Genres_idx"].max() + 1

num_users, num_movies, num_genders, num_ages, num_occupations, num_genres

(6040, 3883, 2, 7, 21, 18)

#### 评分的归一化

In [15]:
min_rating = df["Rating"].min()
max_rating = df["Rating"].max()

df["Rating"] = df["Rating"].map(lambda x : (x-min_rating)/(max_rating-min_rating))

df.sample(frac=1).head(3)

,UserID,MovieID,Rating,Gender,Age,Occupation,UserID_idx,Gender_idx,Age_idx,Occupation_idx,Genres,MovieID_idx,Genres_idx
322748,5963,357,0.75,M,25,15,5962,1,2,2,Comedy,353,0
210068,3081,2115,1.00,M,25,7,3080,1,2,3,Action,2046,3
638152,796,942,1.00,M,50,6,795,1,4,20,Crime,930,8


#### 构建训练数据集

In [16]:
df_sample = df.sample(frac=0.1)
X = df_sample[["UserID_idx","Gender_idx","Age_idx","Occupation_idx","MovieID_idx","Genres_idx"]]
y = df_sample.pop("Rating")

### 2. 搭建双塔模型并训练

In [17]:
def get_model():
    """函数式API搭建双塔DNN模型"""
    
    # 输入
    user_id = keras.layers.Input(shape=(1,), name="user_id")
    gender = keras.layers.Input(shape=(1,), name="gender")
    age = keras.layers.Input(shape=(1,), name="age")
    occupation = keras.layers.Input(shape=(1,), name="occupation")
    movie_id = keras.layers.Input(shape=(1,), name="movie_id")
    genre = keras.layers.Input(shape=(1,), name="genre")
    
    # user 塔
    user_vector = tf.keras.layers.concatenate([
            layers.Embedding(num_users, 100)(user_id), 
            layers.Embedding(num_genders, 2)(gender), 
            layers.Embedding(num_ages, 2)(age), 
            layers.Embedding(num_occupations, 2)(occupation)
    ])
    user_vector = layers.Dense(32, activation='relu')(user_vector)
    user_vector = layers.Dense(8, activation='relu', 
                               name="user_embedding", kernel_regularizer='l2')(user_vector)

    # movie塔
    movie_vector = tf.keras.layers.concatenate([
        layers.Embedding(num_movies, 100)(movie_id),
        layers.Embedding(num_genres, 2)(genre)
    ])
    movie_vector = layers.Dense(32, activation='relu')(movie_vector)
    movie_vector = layers.Dense(8, activation='relu', 
                                name="movie_embedding", kernel_regularizer='l2')(movie_vector)

    # 每个用户的embedding和item的embedding作点积
    dot_user_movie = tf.reduce_sum(user_vector*movie_vector, axis = 1)
    dot_user_movie = tf.expand_dims(dot_user_movie, 1)

    output = layers.Dense(1, activation='sigmoid')(dot_user_movie)
    
    return keras.models.Model(inputs=[user_id, gender, age, occupation, movie_id, genre], outputs=[output]) 

In [18]:
model = get_model()
model.compile(loss=tf.keras.losses.MeanSquaredError(), 
              optimizer=keras.optimizers.RMSprop())

In [19]:
fit_x_train = [
        X["UserID_idx"], 
        X["Gender_idx"],
        X["Age_idx"],
        X["Occupation_idx"],
        X["MovieID_idx"],
        X["Genres_idx"]
    ]

from datetime import datetime
TIMESTAMP = "{0:%Y-%m-%dT%H-%M-%S/}".format(datetime.now())
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs/logs_"+TIMESTAMP)

history = model.fit(
    x=fit_x_train,
    y=y,
    batch_size=32,
    epochs=5,
    verbose=1,
    callbacks=[tensorboard_callback]
)

Epoch 1/5
3126/3126 [==============================] - 24s 8ms/step - loss: 0.0735
Epoch 2/5
3126/3126 [==============================] - 25s 8ms/step - loss: 0.0564
Epoch 3/5
3126/3126 [==============================] - 26s 8ms/step - loss: 0.0542
Epoch 4/5
3126/3126 [==============================] - 27s 9ms/step - loss: 0.0531
Epoch 5/5
3126/3126 [==============================] - 37s 12ms/step - loss: 0.0525


### 3. 模型的预估-predict

In [20]:
inputs = df.sample(frac=1.0)[
    ["UserID_idx","Gender_idx","Age_idx","Occupation_idx","MovieID_idx", "Genres_idx"]].head(10)

In [21]:
# 对于（用户ID，召回的电影ID列表），计算分数
model.predict([
        inputs["UserID_idx"], 
        inputs["Gender_idx"],
        inputs["Age_idx"],
        inputs["Occupation_idx"],
        inputs["MovieID_idx"],
        inputs["Genres_idx"]
    ])

array([[[0.4734862 ]],

       [[0.8372022 ]],

       [[0.43520924]],

       [[0.66028666]],

       [[0.6903477 ]],

       [[0.9391683 ]],

       [[0.58775586]],

       [[0.7064152 ]],

       [[0.809742  ]],

       [[0.49743214]]], dtype=float32)

#### 模型的保存

In [22]:
model.save("./datas/ml-latest-small/tensorflow_two_tower.h5")

In [23]:
new_model = tf.keras.models.load_model("./datas/ml-latest-small/tensorflow_two_tower.h5")

In [24]:
new_model.predict([
        inputs["UserID_idx"], 
        inputs["Gender_idx"],
        inputs["Age_idx"],
        inputs["Occupation_idx"],
        inputs["MovieID_idx"],
        inputs["Genres_idx"]
    ])

array([[[0.4734862 ]],

       [[0.8372022 ]],

       [[0.43520924]],

       [[0.66028666]],

       [[0.6903477 ]],

       [[0.9391683 ]],

       [[0.58775586]],

       [[0.7064152 ]],

       [[0.809742  ]],

       [[0.49743214]]], dtype=float32)

### 4. 保存模型的embedding可用于召回

#### 得到user embedding

In [25]:
model.input

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'gender:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'age:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'occupation:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'movie_id:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'genre:0' shape=(None, 1) dtype=float32>]

In [26]:
user_layer_model = keras.models.Model(
    inputs=[model.input[0], model.input[1], model.input[2], model.input[3]],
    outputs=model.get_layer("user_embedding").output
)

In [27]:
user_embeddings = []
for index, row in df_user.iterrows():
    user_id = row["UserID"]
    user_input = [
        np.reshape(row["UserID_idx"], [1,1]),
        np.reshape(row["Gender_idx"], [1,1]),
        np.reshape(row["Age_idx"], [1,1]),
        np.reshape(row["Occupation_idx"], [1,1])
    ]
    user_embedding = user_layer_model(user_input)
    
    embedding_str = ",".join([str(x) for x in user_embedding.numpy().flatten()])
    user_embeddings.append([user_id, embedding_str])

In [28]:
df_user_embedding = pd.DataFrame(user_embeddings, columns = ["user_id", "user_embedding"])
df_user_embedding.head()

,user_id,user_embedding
0,1,"0.0,0.0,0.5839009,0.0,1.2885361,0.050975706,0...."
1,2,"0.0,0.0,0.7106226,0.0,0.8912425,0.048077803,0...."
2,3,"0.0,0.0,0.7817477,0.0,0.5513846,0.047130395,0...."
3,4,"0.0,0.0,0.650758,0.0,1.0476055,0.049512904,0.0..."
4,5,"0.0,0.0,0.72312206,0.0,0.75548285,0.048189297,..."


In [29]:
output = "./datas/ml-latest-small/tensorflow_user_embedding.csv"
df_user_embedding.to_csv(output, index=False)

#### 得到movie embedding

In [30]:
model.input

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'gender:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'age:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'occupation:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'movie_id:0' shape=(None, 1) dtype=float32>,
 <tf.Tensor 'genre:0' shape=(None, 1) dtype=float32>]

In [31]:
movie_layer_model = keras.models.Model(
    inputs=[model.input[4], model.input[5]],
    outputs=model.get_layer("movie_embedding").output
)

In [32]:
movie_embeddings = []
for index, row in df_movie.iterrows():
    movie_id = row["MovieID"]
    movie_input = [
        np.reshape(row["MovieID_idx"], [1,1]),
        np.reshape(row["Genres_idx"], [1,1])
    ]
    movie_embedding = movie_layer_model(movie_input)
    
    embedding_str = ",".join([str(x) for x in movie_embedding.numpy().flatten()])
    movie_embeddings.append([movie_id, embedding_str])

In [33]:
df_movie_embedding = pd.DataFrame(movie_embeddings, columns = ["movie_id", "movie_embedding"])
df_movie_embedding.head()

,movie_id,movie_embedding
0,1,"0.0,0.0,0.80466634,0.0,1.3142264,0.0,0.0,0.0"
1,2,"0.0,0.0,0.224655,0.0,0.8400048,0.0,0.0,0.0"
2,3,"0.0,0.0,0.17916784,0.0,0.79720753,0.0,0.0,0.0"
3,4,"0.0,0.0,0.0,0.0,0.55387646,0.0,0.0,0.0"
4,5,"0.0,0.0,0.065912835,0.0,0.68349266,0.0,0.0,0.0"


In [34]:
output = "./datas/ml-latest-small/tensorflow_movie_embedding.csv"
df_movie_embedding.to_csv(output, index=False)